# 데이터 전처리
---
## Garbage In, Garbage Out

## 사이킷런을 이용한 ML 학습하기 전에 데이터에 대해 처리할 기본사항


### 1. 결손값 (NA, NaN, NULL) 값은 허용되지 않는다.
- 대체값을 선정해서 치환
- Drop

### 2. 사이킷런의 머신러닝 알고리즘은 문자열을 입력값으로 받지 않는다.
- 인코딩
- Drop


## 결측치 (누락 데이터) 처리

In [ ]:
# 모듈 로딩
import seaborn as sns
import pandas as pd


In [ ]:
# 데이터 로딩
df = sns.load_dataset('titanic')
df.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
# 결측치 확인
# 기본 요약 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [ ]:
# deck 컬럼에 NAN 개수
nan_deck = df['deck'].value_counts(dropna=False)     # value_counts는 NaN을 제외하고 구하는것이 default
nan_deck

deck
NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: count, dtype: int64

In [ ]:
# 직접적인 방법을 결측치 구하기

df.isnull().sum(axis=0)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [ ]:
## 누락 데이터 삭제
df_thresh = df.dropna(axis=1, thresh=500)   # thresh : NaN의 최소 기준치   -> NaN이 500개 이상인 컬럼을 지운다
df_thresh.isnull().sum(axis=0)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
embark_town      2
alive            0
alone            0
dtype: int64

## *중복데이터 처리*


### **[중복 데이터 확인]**

In [ ]:
# 중복 데이터 생성
df = pd.DataFrame({
    'c1' : ['a', 'a', 'b', 'a', 'b'],
    'c2' : [1, 1, 1, 2, 2],
    'c3' : [1, 1, 2, 2, 2,]
})

df

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [ ]:
# duplicated : 중복값이면 True, 아니면 Flase

# 전체 행에서 각 컬럼별 중복값이 모두 일치하는 행 찾기
df_dup = df.duplicated()

df_dup

0    False
1     True
2    False
3    False
4    False
dtype: bool

In [ ]:
# 특정 컬럼별 중복값 찾기
col_dup = df.c2.duplicated()
col_dup

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool

In [ ]:
df

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [ ]:
# 전체 행에서 중복되는 행을 제거
df2 = df.drop_duplicates()
df2

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [ ]:
# 특정 컬럼은 기준으로 중복되는 행을 제거
df3 = df.drop_duplicates(subset='c2')
df3

,c1,c2,c3
0,a,1,1
3,a,2,2


In [ ]:
# 기준이 되는 컬럼을 여러 개 사용 가능
df4 = df.drop_duplicates(subset=['c2', 'c3'])
df4

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


## 데이터 표준화 (피쳐 스케일링)
- 데이터가 잘 정리된 것 처럼 보여도 서로 다른 단위가 섞여 있거나 같은 대상을 다른 형식으로 표현하는 경우가 의외로 많다

- 동일한 대상을 표현하는 방법에 차이가 있으면 분석(예측) 정확도는 현저히 낮아진다
- 데이터 포맷을 일관성 있게 표준화하는 작업이 필요하다
- 데이터의 출처 또는 데이터에 대한 설명이 반드시 필요하다!!



### **[단위 환산]**

In [ ]:
df = pd.read_csv('./auto-mpg.csv')
df.head()

# '''
#     1. mpg:           연비       continuous (연속값)
#     2. cylinders:     실린더수     multi-valued discrete (이산값)
#     3. displacement:  배기량      continuous
#     4. horsepower:    마력       continuous
#     5. weight:        차중       continuous
#     6. acceleration:  가속       continuous
#     7. model year:    생산연도    multi-valued discrete
#     8. origin:        생산지역    multi-valued discrete (1: USA, 2:EU, 3:JPN)
#     9. car name:      자동차명    string (unique for each instance)

# '''

,18.0,8,307.0,130.0,3504.,12.0,70,1,chevrolet chevelle malibu
0,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
1,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
2,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
3,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
4,15.0,8,429.0,198.0,4341.0,10.0,70,1,ford galaxie 500


In [ ]:
feature_name = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']
df = pd.read_csv('./auto-mpg.csv', header=None)
df.columns = feature_name
df.head()


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [ ]:
# mpg (mile per gallon)  -> kpl (kilometer per liter)
mpg_to_kpl = 1.60934 / 3.78541

# 데이터셋에 kpl 열을 추가
df2 = df.copy()
df2['kpl'] = (df2.mpg * mpg_to_kpl)
df2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.652571
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.377143
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.652571
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.802286
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.227428


### **[자료형 변환]**

In [ ]:
df3 = df.copy()

# 전체 기본 정보
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


In [ ]:
# horsepower 컬럼의 고유값 확인
df3.horsepower.unique()

array(['130.0', '165.0', '150.0', '140.0', '198.0', '220.0', '215.0',
       '225.0', '190.0', '170.0', '160.0', '95.00', '97.00', '85.00',
       '88.00', '46.00', '87.00', '90.00', '113.0', '200.0', '210.0',
       '193.0', '?', '100.0', '105.0', '175.0', '153.0', '180.0', '110.0',
       '72.00', '86.00', '70.00', '76.00', '65.00', '69.00', '60.00',
       '80.00', '54.00', '208.0', '155.0', '112.0', '92.00', '145.0',
       '137.0', '158.0', '167.0', '94.00', '107.0', '230.0', '49.00',
       '75.00', '91.00', '122.0', '67.00', '83.00', '78.00', '52.00',
       '61.00', '93.00', '148.0', '129.0', '96.00', '71.00', '98.00',
       '115.0', '53.00', '81.00', '79.00', '120.0', '152.0', '102.0',
       '108.0', '68.00', '58.00', '149.0', '89.00', '63.00', '48.00',
       '66.00', '139.0', '103.0', '125.0', '133.0', '138.0', '135.0',
       '142.0', '77.00', '62.00', '132.0', '84.00', '64.00', '74.00',
       '116.0', '82.00'], dtype=object)

In [ ]:
# '?' 관측값 처리 : NaN으로 바꾸고 NaN 처리
import numpy as np

df3.horsepower.replace('?', np.nan, inplace=True)

df3.horsepower.unique()

array(['130.0', '165.0', '150.0', '140.0', '198.0', '220.0', '215.0',
       '225.0', '190.0', '170.0', '160.0', '95.00', '97.00', '85.00',
       '88.00', '46.00', '87.00', '90.00', '113.0', '200.0', '210.0',
       '193.0', nan, '100.0', '105.0', '175.0', '153.0', '180.0', '110.0',
       '72.00', '86.00', '70.00', '76.00', '65.00', '69.00', '60.00',
       '80.00', '54.00', '208.0', '155.0', '112.0', '92.00', '145.0',
       '137.0', '158.0', '167.0', '94.00', '107.0', '230.0', '49.00',
       '75.00', '91.00', '122.0', '67.00', '83.00', '78.00', '52.00',
       '61.00', '93.00', '148.0', '129.0', '96.00', '71.00', '98.00',
       '115.0', '53.00', '81.00', '79.00', '120.0', '152.0', '102.0',
       '108.0', '68.00', '58.00', '149.0', '89.00', '63.00', '48.00',
       '66.00', '139.0', '103.0', '125.0', '133.0', '138.0', '135.0',
       '142.0', '77.00', '62.00', '132.0', '84.00', '64.00', '74.00',
       '116.0', '82.00'], dtype=object)

In [ ]:
# '?' -> nan -> 삭제 처리
df3.dropna(subset=['horsepower'], axis=0, inplace=True)

df3.horsepower.unique()

array(['130.0', '165.0', '150.0', '140.0', '198.0', '220.0', '215.0',
       '225.0', '190.0', '170.0', '160.0', '95.00', '97.00', '85.00',
       '88.00', '46.00', '87.00', '90.00', '113.0', '200.0', '210.0',
       '193.0', '100.0', '105.0', '175.0', '153.0', '180.0', '110.0',
       '72.00', '86.00', '70.00', '76.00', '65.00', '69.00', '60.00',
       '80.00', '54.00', '208.0', '155.0', '112.0', '92.00', '145.0',
       '137.0', '158.0', '167.0', '94.00', '107.0', '230.0', '49.00',
       '75.00', '91.00', '122.0', '67.00', '83.00', '78.00', '52.00',
       '61.00', '93.00', '148.0', '129.0', '96.00', '71.00', '98.00',
       '115.0', '53.00', '81.00', '79.00', '120.0', '152.0', '102.0',
       '108.0', '68.00', '58.00', '149.0', '89.00', '63.00', '48.00',
       '66.00', '139.0', '103.0', '125.0', '133.0', '138.0', '135.0',
       '142.0', '77.00', '62.00', '132.0', '84.00', '64.00', '74.00',
       '116.0', '82.00'], dtype=object)

In [ ]:
# 현재 object인 horsepower 컬럼 자료형을 수치형(실수형)으로 변환
df3.horsepower = df3.horsepower.astype('float')

df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        392 non-null    float64
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    int64  
 8   car name      392 non-null    object 
dtypes: float64(5), int64(3), object(1)
memory usage: 30.6+ KB


In [ ]:
df3.horsepower.unique()

array([130., 165., 150., 140., 198., 220., 215., 225., 190., 170., 160.,
        95.,  97.,  85.,  88.,  46.,  87.,  90., 113., 200., 210., 193.,
       100., 105., 175., 153., 180., 110.,  72.,  86.,  70.,  76.,  65.,
        69.,  60.,  80.,  54., 208., 155., 112.,  92., 145., 137., 158.,
       167.,  94., 107., 230.,  49.,  75.,  91., 122.,  67.,  83.,  78.,
        52.,  61.,  93., 148., 129.,  96.,  71.,  98., 115.,  53.,  81.,
        79., 120., 152., 102., 108.,  68.,  58., 149.,  89.,  63.,  48.,
        66., 139., 103., 125., 133., 138., 135., 142.,  77.,  62., 132.,
        84.,  64.,  74., 116.,  82.])

## 데이터 정규화

- 각 컬럼(특징, 변수)에 들어있는 숫자데이터의 상대적 크기 차이는 특정 분석(알고리즘) 방법에 따라서 분석 결과를 달라지게 할 수 있다.
- 상대적 크기 차이를 제거하기 위해 데이터 값을 동일한 크기 기준으로 나누는 작업을 정규화(normalization)이라고 한다.
- 보통 정규화 과정을 거친 데이터으이 범위는 0 ~ 1 사이 또는 -1 ~ 1사이가 된다.

---
* [참고] 경우에 따라서는 특정 알고리즘이 상대적 크기 차이를 가정하고 생성되어 있다면 상대적 크기 차이를 갖도록 데이터를 제공하는 것이 더 좋은 결과를 얻을 수 있다.

### **[정규화 1. 해당 컬럼의 최대값(의 절대값)으로 나누기]**


In [ ]:
df4 = df3.copy()

In [ ]:
# 기술통계
df3.horsepower.describe()


count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64

In [ ]:
df3.horsepower = df4.horsepower / abs(df4.horsepower.max())

df3.horsepower.describe()

count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64

### **[정규화2. 각 컬럼 데이터 중에서 최대값과 최소값을 뺀 값으로 나누는 방법]**

In [ ]:
'''
# 각 컬럼 데이터에서 해당 열의 최소값을 뺸 값을 분자,
# 해당 열의 최대값과 최소값의 차이를 분모
# 가장 큰값은 1이 됨
'''

'\n# 각 컬럼 데이터에서 해당 열의 최소값을 뺸 값을 분자,\n# 해당 열의 최대값과 최소값의 차이를 분모\n# 가장 큰값은 1이 됨\n'

In [ ]:
numerator = df4.horsepower - df4.horsepower.min()
denominator = df4.horsepower.max() - df4.horsepower.min()

df4.horsepower = numerator / denominator

df4.horsepower.describe()

count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64

## 범주형(카테고리) 데이터 처리

- 알고리즘에 따라서는  연속데이터를 그대로 사용하기 보다는 일정한 구간(bin)으로 나눠서 분석(학습)하는 것이 효율적인 경우가 있음.

- ex) 가격, 비용, 효율 등의 의미를 가지고 있는 연속적인 값은 일정한 수준이나 정도를 가지고 있는 이산적인 값으로 나타내어서 수준의 차이를 드러내는 방식으로 구현하면 더 효율적인 경우가 있다.

- 이런 과정을 구간 분할(bining)이라고 한다.


- ```
pandas : cut()
```





### **[구간분할]**


In [ ]:
df2.horsepower.replace('?', np.nan, inplace=True)
df2.dropna(subset=['horsepower'], axis=0, inplace=True)
df2.horsepower = df2.horsepower.astype('float')
df2.horsepower.unique()


array([130., 165., 150., 140., 198., 220., 215., 225., 190., 170., 160.,
        95.,  97.,  85.,  88.,  46.,  87.,  90., 113., 200., 210., 193.,
       100., 105., 175., 153., 180., 110.,  72.,  86.,  70.,  76.,  65.,
        69.,  60.,  80.,  54., 208., 155., 112.,  92., 145., 137., 158.,
       167.,  94., 107., 230.,  49.,  75.,  91., 122.,  67.,  83.,  78.,
        52.,  61.,  93., 148., 129.,  96.,  71.,  98., 115.,  53.,  81.,
        79., 120., 152., 102., 108.,  68.,  58., 149.,  89.,  63.,  48.,
        66., 139., 103., 125., 133., 138., 135., 142.,  77.,  62., 132.,
        84.,  64.,  74., 116.,  82.])

In [ ]:
# 3개의 구간으로 나누어서 표기
# np.histogram()

count, bin_dividers = np.histogram(df2.horsepower, bins=3)

print(count)  # 구간별 데이터의 수
print('-'*50)
print(bin_dividers)  # 구간 경계값들



[257 103  32]
--------------------------------------------------
[ 46.         107.33333333 168.66666667 230.        ]


In [ ]:
# 나눠지는 경계로 구간명을 지정
bin_names = ['저출력', '보통출력', '고출력']

In [ ]:
# 각 데이터를 각 구간에 할당 처리

df2['hp_bins'] = pd.cut(x=df2.horsepower,    # 원 데이터 배열
                        bins=bin_dividers,    # 경계 값 리스트
                        labels=bin_names,      # 구간의 이름
                        include_lowest=True)    # 첫 경계값 포함 여부

df2[['horsepower', 'hp_bins']].head(15)

,horsepower,hp_bins
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


In [ ]:
df2.head(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,kpl,hp_bins
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.652571,보통출력
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.377143,보통출력
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.652571,보통출력
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.802286,보통출력
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.227428,보통출력
5,15.0,8,429.0,198.0,4341.0,10.0,70,1,ford galaxie 500,6.377143,고출력
6,14.0,8,454.0,220.0,4354.0,9.0,70,1,chevrolet impala,5.952000,고출력
7,14.0,8,440.0,215.0,4312.0,8.5,70,1,plymouth fury iii,5.952000,고출력
8,14.0,8,455.0,225.0,4425.0,10.0,70,1,pontiac catalina,5.952000,고출력
9,15.0,8,390.0,190.0,3850.0,8.5,70,1,amc ambassador dpl,6.377143,고출력


## **[더미변수]**

- 위 hp_bins 범주형 데이터는 문자열이므로 컴퓨터(모델)이 인식할 수 없는 형태
- 특정 알고리즘은 더 높은 숫자값에 가중치를 부여하는 경우가 있으므로 주의가 필요.
- pandas는 더미변수라는 것을 이용.


- 어떤 특성이 있는지 여부만 표현 => **one-hot encoding**

In [ ]:
# one-hot encoding

# hp_bins 컬럼(번주형데이터)를 더미 변수로 변환
df2 = pd.get_dummies(df2, columns=['hp_bins'], dtype='int')
df2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,kpl,hp_bins_저출력,hp_bins_보통출력,hp_bins_고출력
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.652571,0,1,0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.377143,0,1,0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.652571,0,1,0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.802286,0,1,0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.227428,0,1,0


## 데이터 인코딩

### **[레이블 인코딩]**
- 카테고리 피처를 코드형 숫자값으로 변환하는 방식

In [1]:
items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

In [4]:
from sklearn.preprocessing import LabelEncoder

# Encoder 객체를 생성, fit() - transform()으로 label encoding 수행
encoder = LabelEncoder()

encoder.fit(items)

labels = encoder.transform(items)

print('인코딩 변환값 : ', labels)
print('-'*50)
print('인코딩 클래스 : ', encoder.classes_)


인코딩 변환값 :  [0 1 4 5 3 3 2 2]
--------------------------------------------------
인코딩 클래스 :  ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


In [5]:
# 원본 정보를 가지고 있으므로 디코딩 가능
print('디코딩 클래스 : ', encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

디코딩 클래스 :  ['전자레인지' '컴퓨터' '믹서' 'TV' '냉장고' '냉장고' '선풍기' '선풍기']


In [ ]:
'''
[주의]
# 레이블 인코딩은 lavel 숫자가 1씩 증가되는 형태로 변환되는 특성이 있음
# 연속형 숫자로 이뤄지는 특성에 영향을 받는 모델일 경우에는 lavel encoding이 문제가 될 수 있다

# => one-hot encoding 사용

'''

### **[one-hot encoding]**

- 유의사항
  1. 모든 문자열의 값은 숫자형 값이어야 한다
  2. 입력값으로 2차원의 데이터이어야 한다.

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [8]:
items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

# 숫자값 변환을 위해 Lavelencoder로 변환이 먼저
encoder = LabelEncoder()
labels = encoder.fit_transform(items)
print(f'인코딩 변환값 :  {labels}')

인코딩 변환값 :  [0 1 4 5 3 3 2 2]


In [9]:
# 인코딩 변환값을 2차원 데이터로 변환
labels = labels.reshape(-1, 1)
labels

array([[0],
       [1],
       [4],
       [5],
       [3],
       [3],
       [2],
       [2]])

In [16]:
# one-hot encoding
oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)

print(oh_labels.toarray())
print(f'원핫 인코딩 데이터 차원', oh_labels.shape)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원핫 인코딩 데이터 차원 (8, 6)


In [19]:
# 같은 결과를 pandas로 얻어낼 수 있음
df = pd.DataFrame({'item' : items})

pd.get_dummies(df, dtype='int')

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


## 피쳐스케일링(Feature Scaling)

- 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업
  - 피쳐스케일링의 대표적 방법 : 표준화(Standardixation), 정규화(Normalixation)

In [ ]:
'''
# 표준화 : 데이터셋의 피처 각각이 평균이 0이고 분산이 1인 가우시안 정규분호를 가진 값으로 변환하는 것
# 정규화 : 서로 다른 피처 크기를 통이시키기 위해 그 크기를 변환해주는 것

# => 사이킷런은 이런 작업을 통칭해서 피처스케일링

'''

### **[StandardScaler]**

In [24]:
from sklearn.datasets import load_iris

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [25]:
print('=== feature 들의 평균 값 ===')
print(iris_df.mean())
print('=== feature 들의 분산 값 ===')
print(iris_df.var())

=== feature 들의 평균 값 ===
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64
=== feature 들의 분산 값 ===
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [26]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
scaler = StandardScaler()

# StandardScaler로 데이터 세트 변환 : fit(), transform()
scaler.fit(iris_df)

iris_scaled = scaler.transform(iris_df)

iris_scaled

array([[-9.00681170e-01,  1.01900435e+00, -1.34022653e+00,
        -1.31544430e+00],
       [-1.14301691e+00, -1.31979479e-01, -1.34022653e+00,
        -1.31544430e+00],
       [-1.38535265e+00,  3.28414053e-01, -1.39706395e+00,
        -1.31544430e+00],
       [-1.50652052e+00,  9.82172869e-02, -1.28338910e+00,
        -1.31544430e+00],
       [-1.02184904e+00,  1.24920112e+00, -1.34022653e+00,
        -1.31544430e+00],
       [-5.37177559e-01,  1.93979142e+00, -1.16971425e+00,
        -1.05217993e+00],
       [-1.50652052e+00,  7.88807586e-01, -1.34022653e+00,
        -1.18381211e+00],
       [-1.02184904e+00,  7.88807586e-01, -1.28338910e+00,
        -1.31544430e+00],
       [-1.74885626e+00, -3.62176246e-01, -1.34022653e+00,
        -1.31544430e+00],
       [-1.14301691e+00,  9.82172869e-02, -1.28338910e+00,
        -1.44707648e+00],
       [-5.37177559e-01,  1.47939788e+00, -1.28338910e+00,
        -1.31544430e+00],
       [-1.26418478e+00,  7.88807586e-01, -1.22655167e+00,
      

In [27]:
# 피쳐스케일링 결과가 ndarray 객체로 리턴된다

iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
iris_df_scaled

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444
...,...,...,...,...
145,1.038005,-0.131979,0.819596,1.448832
146,0.553333,-1.282963,0.705921,0.922303
147,0.795669,-0.131979,0.819596,1.053935
148,0.432165,0.788808,0.933271,1.448832


In [29]:
print('=== feature 들의 평균 값 ===')
print(iris_df_scaled.mean())
print('=== feature 들의 분산 값 ===')
print(iris_df_scaled.var())

=== feature 들의 평균 값 ===
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64
=== feature 들의 분산 값 ===
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


### **[MinMaxScaler]**

- 데이터 값을 0과1사이의 범위 값으로 변환 (음수값이 있으면 -1부터 1값으로 변환)
- 데이터 분포 가우시안 분포가 아닐 경우에 적용해 볼 수 있음

In [30]:
from sklearn.preprocessing import MinMaxScaler

# 객체 생성
scaler = MinMaxScaler()

# fit(), transfrom()
scaler.fit(iris_df)

iris_scaled = scaler.transform(iris_df)

In [32]:
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)


print('=== feature 들의 최소 값 ===')
print(iris_df_scaled.min())
print('=== feature 들의 최대 값 ===')
print(iris_df_scaled.max())

=== feature 들의 최소 값 ===
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64
=== feature 들의 최대 값 ===
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


## **[유의사항]**

- Scaler를 이용해서 학습데이터와 테스트데이터에 fit(), transform(), fit_transfrom() 적용시
- 학습데이터로 fit()을 수행한 스케일링 기준 정보를 그대로 테스트 데이터에 적용해야 한다


In [34]:
# 문제상황

from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [35]:
# 학습데이터는 0~10까지 , 테스트데이터 : 0~5 값을 갖는 데이터 세트
# fit(), transform() 2차원 이상의 데이터 형태만 가능함

train_array = np.arange(0, 11).reshape(-1, 1)
test_array = np.arange(0, 6).reshape(-1, 1)

In [37]:
# 피처 스케일링 적용

# 객체 호출
scaler = MinMaxScaler()

# fit() - 기준 적용 (최소값을 0, 최대값을 10으로 설정)
scaler.fit(train_array)

# transform() - 1/10 scale 적용
train_scaled = scaler.transform(train_array)

# 데이터 확인
print('원본 train_array 데이터 값: ', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터 값: ', np.round(train_scaled.reshape(-1), 2))


원본 train_array 데이터 값:  [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터 값:  [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [38]:
# 피쳐 스케일링 (test data) 적용

# 객체 호출
scaler = MinMaxScaler()

# fit() - 기준 적용 (최솟값을 0, 최대값을 5) => 새로운 기준 정보가 생성되었다   -  문제점!
scaler.fit(test_array)

# transform()
test_scaled = scaler.transform(test_array)

# 데이터 확인
print('원본 test_array 데이터 값: ', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터 값: ', np.round(test_scaled.reshape(-1), 2))

원본 test_array 데이터 값:  [0 1 2 3 4 5]
Scale된 test_array 데이터 값:  [0.  0.2 0.4 0.6 0.8 1. ]


In [39]:
# 올바른 방법
scaler = MinMaxScaler()

# fit() - 기준 적용 (최소값을 0, 최대값을 10으로 설정)
scaler.fit(train_array)

# transform()
train_scaled = scaler.transform(train_array)

# 데이터 확인
print('원본 train_array 데이터 값: ', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터 값: ', np.round(train_scaled.reshape(-1), 2))



# 테스트 데이터의 scale 변환은 fit() 호출없이 train 데이터의 기준을 적용해서 바로 transfrom()만 수행
test_scaled = scaler.transform(test_array)


# 데이터 확인
print('원본 test_array 데이터 값: ', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터 값: ', np.round(test_scaled.reshape(-1), 2))


원본 train_array 데이터 값:  [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터 값:  [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
원본 test_array 데이터 값:  [0 1 2 3 4 5]
Scale된 test_array 데이터 값:  [0.  0.1 0.2 0.3 0.4 0.5]
